# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = -0.02
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
ROBOT_Z_ANGLE = np.pi
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
# add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

add_kiro_indytool_down(gscene, zoff=TOOL_OFFSET, tool_link=TIP_LINK, face_name=TOOL_NAME)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

Current PC IP: 192.168.0.123
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
[MOBILE ROBOT] bind: ('192.168.0.123', 50306)
[MOBILE ROBOT] Start UDP THREAD
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., -50.,  70.,  -0.,  90., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

#### 1.1.2  detect bed and add to the scene

In [3]:
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_leftup, closet_rightup, closet_down = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1.5,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

### 1.2 Make bed cleaning plan

#### 1.2.1 get division-base pose combination data

In [4]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [5]:
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

In [6]:
BED_OFFSET = 0.0
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(0,0,-gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE-BED_OFFSET), 
                                  rpy=(0,0,0))

T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]

In [7]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, tip_dir=None, TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02, 
                                          sweep_margin=0.0, xout_cut=True)

('Height Reference: ', 0.5320000240802765)


#### 1.2.2 select base poses and generate motions

In [8]:
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
test_fun = TestBaseDivFunc(ppline, floor_ws, bed_mat, WP_DIMS, TOOL_DIM, crob.home_dict)

In [9]:
test_fun.clear()

idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step,
    test_fun=test_fun)

[0.32, 0.08]
[-0.028  0.295]
[-0.86   0.295]
0
[0.32, 0.08]
1
Use 36/36 agents
Goal reached
======================= terminated 10: first answer acquired ===============================
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 35: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 24: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 30: first answer acquired from other agent ===============================
======================= terminated 32: first an

======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
Goal reached
======================= terminated 6: first answer acquired ===============================
======================= terminated 25: first answer acquired from other agent ===============================
Goal reached
======================= terminated 23: first answer acquired ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
======================= terminated 5: first answer acquired from other agent ===============================
Goal reached
========

======================= terminated 33: first answer acquired from other agent ===============================
Goal reached
======================= terminated 0: first answer acquired ===============================
Goal reached
======================= terminated 28: first answer acquired ===============================
======================= terminated 11: first answer acquired from other agent ===============================
======================= terminated 17: first answer acquired from other agent ===============================
Goal reached
======================= terminated 27: first answer acquired ===============================
======================= terminated 16: first answer acquired from other agent ===============================
Goal reached
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 24: first answer acquired ===============================
======================= term

======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 29: first answer acquired from other agent ===============================
======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired from other agent ===============================
======================= terminated 27: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
============

Goal reached
======================= terminated 14: first answer acquired ===============================
======================= terminated 26: first answer acquired from other agent ===============================
Goal reached
======================= terminated 19: first answer acquired ===============================
Goal reached
======================= terminated 17: first answer acquired ===============================
Goal reached
======================= terminated 12: first answer acquired ===============================
Goal reached
======================= terminated 25: first answer acquired ===============================
[0.32, 0.08]
[0.86 0.  ]
[0.028 0.   ]
0
[0.32, 0.08]
1
Use 36/36 agents
======================= terminated 30: max iteration time reached (1633647704/1633647701.17 s) ===============================
======================= terminated 32: max iteration time reached (1633647704/1633647701.17 s) ===============================
======================= terminate

...

Goal reached
======================= terminated 6: first answer acquired ===============================
Goal reached
======================= terminated 32: first answer acquired ===============================
Goal reached
======================= terminated 33: first answer acquired ===============================
Goal reached
======================= terminated 31: first answer acquired ===============================
Goal reached
Goal reached
======================= terminated 22: first answer acquired ===============================
Goal reached
======================= terminated 25: first answer acquired ===============================
======================= terminated 26: first answer acquired ===============================
Goal reached
======================= terminated 30: first answer acquired ===============================
Goal reached
======================= terminated 35: first answer acquired ===============================
Goal reached
======================= terminated

======================= terminated 19: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 16: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 2: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 25: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 28: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 22: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 35: max iteration time reached (1633647713/1633647710.92 s) ===============================
======================= terminated 3: max iteration time reached (1633647713/1633647710.92 s) ==================

======================= terminated 20: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 35: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 22: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 5: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 4: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 24: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 32: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 13: max iteration time reached (1633647719/1633647716.09 s) ==================

======================= terminated 11: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 33: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 0: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 1: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 17: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 29: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 25: max iteration time reached (1633647719/1633647716.09 s) ===============================
======================= terminated 12: max iteration time reached (1633647719/1633647716.09 s) ==================

======================= terminated 8: first answer acquired from other agent ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
======================= terminated 31: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired from other agent ===============================
==============

#### 1.2.3 refine motions

In [10]:
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list, scene_args_list, scene_kwargs_list = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, bed_mat, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

[0.32, 0.08]
[-0.028  0.295]
[-0.86   0.295]
0
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[-0.028  0.   ]
[-0.86  0.  ]
0
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[0.86  0.295]
[0.028 0.295]
0
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[0.86 0.  ]
[0.028 0.   ]
0
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[ 0.86  -0.295]
[ 0.62  -0.295]
0
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[ 0.564 -0.295]
[-0.268 -0.295]
0
[0.32, 0.08]
1
Goal reached


### 1.3 Execute bed cleaning sequence

In [11]:
# indy.traj_vel = 3

In [12]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt], period=0.002)

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

[0.32, 0.08]
[-0.028  0.295]
[-0.86   0.295]
0
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [ 0.   -0.04 -0.41  0.58]
target xyzw: [ 0.   -0.51 -0.41  0.58]
move to: [ 0.   -0.51 -0.41  0.58]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[-0.028  0.   ]
[-0.86  0.  ]
0
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.2 -0.   0.   1. ]
target xyzw: [0.3 0.  0.  1. ]
move to: [0.3 0.  0.  1. ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[0.86  0.295]
[0.028 0.295]
0
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.79  0.89  0.    1.  ]
target xyzw: [-0.29  0.89  0.    1.  ]
move to: [-0.15  0.44  0.    1.  ]
move to: [-0.29  0.89  0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
[0.32, 0.08]
[0.86 0.  ]
[0.028 0.   ]
0
[0.32, 0.08]
1
curre  xyzw: [0 0 0 1]
ready  xyzw: [ 2.65 -0.3   1.   -0.  ]
target xyzw: [ 2.15 -0.3   1.   -0.  ]
move to: [ 2.15 -0.3   1.   -0.  ]
move to: [ 2.15 -0.3   1.   -0.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1

In [13]:
test_fun.clear()
for swp_fin in swp_fin_list:
    gscene.remove(swp_fin)
swp_fin_list = []
pscene.clear_subjects()
for child in copy.copy(bed_mat.children):
    gscene.remove(gscene.NAME_DICT[child])


## 2. Closet cleaning

### 2.2. Make closet cleaning plan

#### 2.2.1 make plans

In [23]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [24]:
wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))

brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(0,0,-gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
                                  rpy=(0,0,0))
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    


In [25]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_leftup, brush_face, robot_config, 
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_leftup, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=1)
test_fun_cl.clear()
idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_leftup, scene_args_list_leftup, scene_kwargs_list_leftup = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, closet_leftup, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

[0.32, 0.08]
[0.49   0.0875]
[ 0.49   -0.0875]
1
[0.32, 0.08]
1
Use 36/36 agents
======================= terminated 2: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 12: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 13: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 21: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 5: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 16: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 11: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminat

======================= terminated 18: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 7: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 28: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 17: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 19: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 1: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 24: max iteration time reached (1633647871/1633647868.02 s) ===============================
======================= terminated 27: max iteration time reached (1633647871/1633647868.02 s) ==================

Goal reached
======================= terminated 4: first answer acquired ===============================
======================= terminated 3: first answer acquired from other agent ===============================
Goal reached
======================= terminated 10: first answer acquired ===============================
======================= terminated 23: first answer acquired from other agent ===============================
Goal reached
======================= terminated 28: first answer acquired ===============================
Goal reached
======================= terminated 13: first answer acquired ===============================
======================= terminated 26: first answer acquired from other agent ===============================
Goal reached
======================= terminated 32: first answer acquired from other agent ===============================
======================= terminated 14: first answer acquired ===============================
Goal reached
===================

======================= terminated 32: first answer acquired from other agent ===============================
======================= terminated 26: first answer acquired from other agent ===============================
======================= terminated 25: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
Goal reached
======================= terminated 15: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired ===============================
======================= terminated 34: first answer acquired from other agent ===============================
======================= terminated 20: first answer acquired from other agent ===============================
================

======================= terminated 17: first answer acquired ===============================
Goal reached
======================= terminated 12: first answer acquired ===============================
======================= terminated 27: first answer acquired from other agent ===============================
======================= terminated 30: first answer acquired from other agent ===============================
======================= terminated 3: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired from other agent ===============================
[0.32, 0.08]
[0.     0.0875]
[ 0.     -0.0875]
1
[0.32, 0.08]
1
Use 36/36 agents
Goal reached
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired ===============================
======================= terminated 12: first answer acquired from ot

======================= terminated 21: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 4: first answer acquired ===============================
======================= terminated 32: first answer acquired from other agent ===============================
Goal reached
======================= terminated 7: first answer acquired ===============================
Goal reached
======================= terminated 14: first answer acquired ===============================
Goal reached
======================= terminated 10: first answer acquired ===============================
======================= terminated 22: first answer acquired from other agent ===============================
Goal reached
======================= terminated 15: first answer acquired ===============================
Goal reached
======================= terminat

======================= terminated 28: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
Goal reached
======================= terminated 10: first answer acquired ===============================
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ===============================
Goal reached
======================= terminated 5: first answer acquired from other agent ===============================
======================= terminated 33: first answer acquired ===============================
======================= terminated 15: first answer acquired from other agent ===============================
===================

======================= terminated 1: first answer acquired from other agent ===============================
[0.32, 0.08]
[-0.245   0.0875]
[-0.245  -0.0875]
1
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[0.     0.0875]
[ 0.     -0.0875]
1
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[0.245  0.0875]
[ 0.245  -0.0875]
1
[0.32, 0.08]
1
Goal reached
[0.32, 0.08]
[0.49   0.0875]
[ 0.49   -0.0875]
1
[0.32, 0.08]
1
Goal reached


In [26]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_rightup, brush_face, robot_config, 
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_rightup, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=1)
test_fun_cl.clear()
idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_rightup, scene_args_list_rightup, scene_kwargs_list_rightup = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, closet_rightup, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

[0.32, 0.08]
[0.115 0.06 ]
[ 0.115 -0.06 ]
1
[0.32, 0.08]
1
Use 36/36 agents
Goal reached
======================= terminated 2: first answer acquired ===============================
Goal reached
Goal reached
======================= terminated 28: first answer acquired from other agent ===============================
Goal reached
======================= terminated 6: first answer acquired ===============================
======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired ===============================
======================= terminated 14: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired ===============================
======================= terminated 29: first answer acquired f

======================= terminated 31: first answer acquired from other agent ===============================
Goal reached
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 23: first answer acquired ===============================
Goal reached
======================= terminated 15: first answer acquired ===============================
======================= terminated 35: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
Goal reached
======================= terminated 17: first answer acquired ===============================
======================= terminated 26: first answer acquired from other agent ===============================
======================= terminated 0: first answer acquired from other agent ===============================
======================= 

In [ ]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_down, brush_face, robot_config, 
                                          plane_val=None, tip_dir="down", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_down, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=-1)
test_fun_cl.clear()

idx_bases, idc_divs, covered_all, snode_schedule_list = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_down, scene_args_list_down, scene_kwargs_list_down = refine_order_plan(
    ppline, snode_schedule_list, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, closet_down, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)
test_fun_cl.clear()

[0.32, 0.08]
[0.24   0.2075]
[ 0.24   -0.2075]
1
[0.32, 0.08]
1
Use 36/36 agents


In [ ]:
snode_schedule_list = snode_schedule_list_leftup + snode_schedule_list_rightup + snode_schedule_list_down
scene_args_list = scene_args_list_leftup + scene_args_list_rightup + scene_args_list_down
scene_kwargs_list = scene_kwargs_list_leftup + scene_kwargs_list_rightup + scene_kwargs_list_down

### 2.3 Execute close cleaning sequence

In [ ]:
# indy.traj_vel = 3

In [ ]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

In [ ]:
test_fun_cl.clear()
for swp_fin in swp_fin_list:
    gscene.remove(swp_fin)
swp_fin_list = []
pscene.clear_subjects()
for child in copy.copy(closet_leftup.children):
    gscene.remove(gscene.NAME_DICT[child])
for child in copy.copy(closet_rightup.children):
    gscene.remove(gscene.NAME_DICT[child])
for child in copy.copy(closet_down.children):
    gscene.remove(gscene.NAME_DICT[child])

## 3. Return to home